In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
import base64
import pandas
from sklearn.linear_model import SGDClassifier

In [2]:
def get_data(data_file_path):
    data = pandas.read_csv(data_file_path, sep='\t')
    data['Html'] = data['PageBase64'].apply(base64.b64decode)
    return data.drop(['PageBase64', 'Url'], axis=1)

In [3]:
train_data = get_data('data/kaggle_train_data_tab.csv')
test_data = get_data('data/kaggle_test_data_tab.csv')
vectorizer = TfidfVectorizer(decode_error='ignore')

In [4]:
train_data.head()

,Id,Prediction,Html
0,-9222401963271173253,0,"b'<!DOCTYPE html PUBLIC ""-//W3C//DTD XHTML 1.0..."
1,-9221399504663603656,0,"b'<!DOCTYPE html PUBLIC ""-//W3C//DTD XHTML 1.0..."
2,-9219733223042265364,0,"b'<!DOCTYPE HTML PUBLIC ""-//W3C//DTD HTML 4.01..."
3,-9218561774155397294,0,"b'<!DOCTYPE HTML PUBLIC ""-//W3C//DTD HTML 4.0 ..."
4,-9216163564171210203,1,"b'<!DOCTYPE html PUBLIC ""-//W3C//DTD XHTML 1.0..."


In [5]:
test_data.head()

,Id,Prediction,Html
0,-9219333997282093076,-1,"b'<html>\r\n\r\n<head>\r\n<META content=""text/..."
1,-9216796593526314243,-1,"b'<!DOCTYPE html PUBLIC ""-//W3C//DTD XHTML 1.0..."
2,-9216634615606576667,-1,b'<!--\xd0\x98\xd0\xb4\xd0\xb5\xd1\x8f \xd1\x8...
3,-9215626373068452590,-1,"b'\n<!DOCTYPE HTML PUBLIC ""-//W3C//DTD HTML 4...."
4,-9215403080479774736,-1,b'\r\n\r\n\r\n<HTML>\r\n<HEAD>\r\n<title>\xd0\...


In [6]:
tf_idf_train = vectorizer.fit_transform(train_data.Html)

In [10]:
classifier = SGDClassifier()
classifier.fit(tf_idf_train, train_data.Prediction)

SGDClassifier(alpha=0.0001, average=False, class_weight=None,
              early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
              l1_ratio=0.15, learning_rate='optimal', loss='hinge',
              max_iter=1000, n_iter_no_change=5, n_jobs=None, penalty='l2',
              power_t=0.5, random_state=None, shuffle=True, tol=0.001,
              validation_fraction=0.1, verbose=0, warm_start=False)

In [11]:
tf_idf_test = vectorizer.transform(test_data.Html)

In [12]:
test_data.Prediction = classifier.predict(tf_idf_test)

In [13]:
test_data.head()

,Id,Prediction,Html
0,-9219333997282093076,1,"b'<html>\r\n\r\n<head>\r\n<META content=""text/..."
1,-9216796593526314243,1,"b'<!DOCTYPE html PUBLIC ""-//W3C//DTD XHTML 1.0..."
2,-9216634615606576667,0,b'<!--\xd0\x98\xd0\xb4\xd0\xb5\xd1\x8f \xd1\x8...
3,-9215626373068452590,0,"b'\n<!DOCTYPE HTML PUBLIC ""-//W3C//DTD HTML 4...."
4,-9215403080479774736,1,b'\r\n\r\n\r\n<HTML>\r\n<HEAD>\r\n<title>\xd0\...


In [29]:
with open('data/result.csv', 'w') as file:
    file.write('Id,Prediction\n')
    for cur_line in test_data.itertuples():
        file.write("{0},{1}\n".format(cur_line.Id, cur_line.Prediction))